In [35]:
import geopandas as gpd
import os
from glob import glob
import pandas as pd

In [2]:
directory = '../data/aeroDEM'
aerodem_shapefile_path = glob('**/*.shp', root_dir=directory, recursive=True)

aeroDEM_catalog = gpd.read_file(
    os.path.join(directory, aerodem_shapefile_path[0])
    )

centreline = gpd.read_file('../data/streams_v3.geojson')

aeroDEM_catalog = aeroDEM_catalog.to_crs(centreline.crs)

In [3]:
dem_dict = {}
for row in centreline.itertuples():
    dems = aeroDEM_catalog.loc[aeroDEM_catalog.intersects(row.geometry)]
    dem_dict[row.Index] = dems

In [36]:
centreline['utm_zone'] = pd.Series()
for i in range(len(centreline)):
    centreline.loc[i, 'utm_zone'] = centreline.iloc[[i]].estimate_utm_crs().utm_zone

In [41]:
def make_file_paths(year, utm_zone):
    string = f'aerodem_{year}_utm{utm_zone}'
    return string+'.tif', string+'.tfw'

In [42]:
p = 'https://www.ncei.noaa.gov/data/oceans/archive/arc0088/0145405/1.1/data/0-data/G150AERODEM/DEM/aerodem_1978_utm19.tif'

In [45]:
import rioxarray as rio

In [48]:
ds = rio.open_rasterio(p, chunks='auto')

In [49]:
ds

<xarray.DataArray (band: 1, y: 7730, x: 8342)> Size: 258MB
dask.array<open_rasterio-e063c070fb243d727adf996489d626c6<this-array>, shape=(1, 7730, 8342), dtype=float32, chunksize=(1, 5760, 5760), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 67kB 3.982e+05 3.983e+05 ... 6.068e+05 6.068e+05
  * y            (y) float64 62kB 8.831e+06 8.831e+06 ... 8.637e+06 8.637e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:           Area
    TIFFTAG_ARTIST:          KORSGAARD, NJ (2015)
    RepresentationType:      ATHEMATIC
    STATISTICS_COVARIANCES:  76232,4133920755
    STATISTICS_MAXIMUM:      1133,1400146484
    STATISTICS_MEAN:         338,92804886329
    STATISTICS_MINIMUM:      -218,05000305176
    STATISTICS_STDDEV:       276,10217925992
    _FillValue:              -3.4028235e+38
    scale_factor:            1.0
    add_offset:              0.0